In [7]:
# Import basic libraries.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import models.

from prophet import Prophet
from sktime.forecasting.ets import AutoETS # Modelo ETS
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error # Calcular  porcentaje MAE
from sktime.forecasting.exp_smoothing import ExponentialSmoothing # Modelo ETS manual
from sktime.transformations.series.boxcox import LogTransformer # Transforamción Logarítmica
from sktime.forecasting.arima import AutoARIMA # Modelo ARIMA
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.statsforecast import StatsForecastAutoARIMA
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.tbats import TBATS

# Import metrics and utilities.
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sktime.forecasting.base import ForecastingHorizon
from sktime.utils.plotting import plot_series # Graficar series
from sktime.forecasting.model_selection import temporal_train_test_split # Dividir dataset en train y test
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.performance_metrics.forecasting import MeanSquaredError # Calcular MAE
from statsmodels.tsa.seasonal import STL

# Ignore warnings.

import warnings
warnings.filterwarnings('ignore')

In [8]:
transaction_df = pd.read_csv("transactions_data_merged.csv")

In [16]:
transaction_df

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,...,7,Female,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,...,6,Male,604 Pine Street,40.80,-91.12,$18076,$36853,$112139,834,5
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,...,4,Male,2379 Forest Lane,33.18,-117.29,$16894,$34449,$36540,686,3
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,...,5,Female,903 Hill Boulevard,41.42,-87.35,$26168,$53350,$128676,685,5
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,...,5,Male,166 River Drive,38.86,-76.60,$33529,$68362,$96182,711,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,...,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,...,9,Male,6076 Bayview Boulevard,43.06,-87.96,$9995,$20377,$12092,789,4
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,...,4,Female,7927 Plum Lane,33.10,-96.66,$32580,$78329,$40161,720,3
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,...,12,Female,5887 Seventh Lane,37.68,-122.43,$23752,$48430,$62384,716,2


In [14]:
import json

with open("predictions/predictions_4.json", "r") as file:
    data = json.load(file)

# Extraer el diccionario de etiquetas dentro de "target"
labels = data["target"]

In [19]:
# Crea un df nuevo a partir de transaction_df quedandote unicamente con las filas en las que esten los valores de "label" en client_id.
transaction_df_filtrado = transaction_df[transaction_df["client_id"].astype(str).isin(labels)]

In [23]:
transaction_df_filtrado.shape

(13305915, 36)

In [28]:
transaction_df_filtrado

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,...,7,Female,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,...,6,Male,604 Pine Street,40.80,-91.12,$18076,$36853,$112139,834,5
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,...,4,Male,2379 Forest Lane,33.18,-117.29,$16894,$34449,$36540,686,3
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,...,5,Female,903 Hill Boulevard,41.42,-87.35,$26168,$53350,$128676,685,5
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,...,5,Male,166 River Drive,38.86,-76.60,$33529,$68362,$96182,711,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,...,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,...,9,Male,6076 Bayview Boulevard,43.06,-87.96,$9995,$20377,$12092,789,4
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,...,4,Female,7927 Plum Lane,33.10,-96.66,$32580,$78329,$40161,720,3
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,...,12,Female,5887 Seventh Lane,37.68,-122.43,$23752,$48430,$62384,716,2


In [35]:
# Convert the date column to datetime format. Strftime is used to specify the format of the date.

transaction_df_filtrado['date'] = pd.to_datetime(transaction_df_filtrado['date'])

# Monthly format.

transaction_df_filtrado['date'] = transaction_df_filtrado['date'].dt.strftime('%Y-%m')

transaction_df_filtrado_ordenado = transaction_df_filtrado.sort_values(by='date', ascending=True)

transaction_df_filtrado_ordenado

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,...,7,Female,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4
67478,7555949,2010-01,1941,2030,$55.78,Swipe Transaction,43293,Cooper,TX,75432.0,...,8,Male,2867 Federal Drive,33.14,-95.95,$18934,$38609,$27463,740,3
67477,7555948,2010-01,1797,1127,$62.72,Online Transaction,16798,ONLINE,NaN,NaN,...,11,Male,391 Martin Luther King Boulevard,37.71,-122.16,$24971,$30962,$15336,743,5
67476,7555947,2010-01,1670,3304,$73.00,Swipe Transaction,43293,Long Beach,CA,90808.0,...,12,Male,46 Lincoln Lane,33.82,-118.11,$32079,$50184,$27778,732,4
67475,7555945,2010-01,1357,4611,$-56.00,Swipe Transaction,61195,Tacoma,WA,98404.0,...,12,Female,402 Birch Boulevard,47.20,-122.40,$16941,$11410,$0,812,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227659,23665325,2019-10,349,4314,$60.00,Swipe Transaction,27092,Wooster,OH,44691.0,...,12,Female,1943 Birch Street,40.84,-81.63,$21201,$43229,$96831,685,2
13227658,23665324,2019-10,261,30,$10.71,Chip Transaction,75804,Guadalajara,Mexico,NaN,...,2,Male,7061 Littlewood Street,36.00,-114.96,$28985,$59093,$117325,737,5
13227657,23665323,2019-10,201,4489,$14.78,Chip Transaction,88646,Rome,Italy,NaN,...,5,Male,206 Pine Lane,41.52,-88.12,$17567,$35823,$96691,732,3
13227655,23665321,2019-10,1060,4291,$14.23,Chip Transaction,50783,Indianapolis,IN,46222.0,...,2,Male,359 Valley Street,39.77,-86.14,$17909,$21547,$16040,785,5


In [37]:
# SAme of amount to per_capita_income, yearly_income, total_debt.
transaction_df_filtrado_ordenado['per_capita_income'] = transaction_df_filtrado_ordenado['per_capita_income'].str.replace('$', '', regex=False)
transaction_df_filtrado_ordenado['per_capita_income'] = pd.to_numeric(transaction_df_filtrado_ordenado['per_capita_income'], errors='coerce')

transaction_df_filtrado_ordenado['yearly_income'] = transaction_df_filtrado_ordenado['yearly_income'].str.replace('$', '', regex=False)
transaction_df_filtrado_ordenado['yearly_income'] = pd.to_numeric(transaction_df_filtrado_ordenado['yearly_income'], errors='coerce')

transaction_df_filtrado_ordenado['total_debt'] = transaction_df_filtrado_ordenado['total_debt'].str.replace('$', '', regex=False)
transaction_df_filtrado_ordenado['total_debt'] = pd.to_numeric(transaction_df_filtrado_ordenado['total_debt'], errors='coerce')

# Modificamos amount
transaction_df_filtrado_ordenado['amount'] = transaction_df_filtrado_ordenado['amount'].str.replace('$', '', regex=False)
transaction_df_filtrado_ordenado['amount'] = pd.to_numeric(transaction_df_filtrado_ordenado['amount'], errors='coerce')

transaction_df_filtrado_ordenado

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,...,7,Female,594 Mountain View Street,46.80,-100.76,23679,48277,110153,740,4
67478,7555949,2010-01,1941,2030,55.78,Swipe Transaction,43293,Cooper,TX,75432.0,...,8,Male,2867 Federal Drive,33.14,-95.95,18934,38609,27463,740,3
67477,7555948,2010-01,1797,1127,62.72,Online Transaction,16798,ONLINE,NaN,NaN,...,11,Male,391 Martin Luther King Boulevard,37.71,-122.16,24971,30962,15336,743,5
67476,7555947,2010-01,1670,3304,73.00,Swipe Transaction,43293,Long Beach,CA,90808.0,...,12,Male,46 Lincoln Lane,33.82,-118.11,32079,50184,27778,732,4
67475,7555945,2010-01,1357,4611,-56.00,Swipe Transaction,61195,Tacoma,WA,98404.0,...,12,Female,402 Birch Boulevard,47.20,-122.40,16941,11410,0,812,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227659,23665325,2019-10,349,4314,60.00,Swipe Transaction,27092,Wooster,OH,44691.0,...,12,Female,1943 Birch Street,40.84,-81.63,21201,43229,96831,685,2
13227658,23665324,2019-10,261,30,10.71,Chip Transaction,75804,Guadalajara,Mexico,NaN,...,2,Male,7061 Littlewood Street,36.00,-114.96,28985,59093,117325,737,5
13227657,23665323,2019-10,201,4489,14.78,Chip Transaction,88646,Rome,Italy,NaN,...,5,Male,206 Pine Lane,41.52,-88.12,17567,35823,96691,732,3
13227655,23665321,2019-10,1060,4291,14.23,Chip Transaction,50783,Indianapolis,IN,46222.0,...,2,Male,359 Valley Street,39.77,-86.14,17909,21547,16040,785,5


In [38]:
transaction_df_filtrado_ordenado.dtypes

transactions_id            int64
date                      object
client_id                  int64
card_id                    int64
amount                   float64
use_chip                  object
merchant_id                int64
merchant_city             object
merchant_state            object
zip                      float64
mcc                        int64
errors                    object
card_brand                object
card_type                 object
card_number                int64
expires                   object
cvv                        int64
has_chip                  object
num_cards_issued           int64
credit_limit              object
acct_open_date            object
year_pin_last_changed      int64
card_on_dark_web          object
current_age                int64
retirement_age             int64
birth_year                 int64
birth_month                int64
gender                    object
address                   object
latitude                 float64
longitude 

In [102]:
# Select only the columns date, client_id, amount, per_capita_income, yearly_income, total_debt, num_credit_cards.

transaction_df_filtrado_ordenado_selected = transaction_df_filtrado_ordenado[['date', 'client_id', 'amount', 'per_capita_income', 'yearly_income', 'total_debt', 'num_credit_cards']]
transaction_df_filtrado_ordenado_selected

,date,client_id,amount,per_capita_income,yearly_income,total_debt,num_credit_cards
0,2010-01,1556,-77.00,23679,48277,110153,4
67478,2010-01,1941,55.78,18934,38609,27463,3
67477,2010-01,1797,62.72,24971,30962,15336,5
67476,2010-01,1670,73.00,32079,50184,27778,4
67475,2010-01,1357,-56.00,16941,11410,0,7
...,...,...,...,...,...,...,...
13227659,2019-10,349,60.00,21201,43229,96831,2
13227658,2019-10,261,10.71,28985,59093,117325,5
13227657,2019-10,201,14.78,17567,35823,96691,3
13227655,2019-10,1060,14.23,17909,21547,16040,5


In [103]:
# Crea una variable que sea Inflows y Expenses Para los que son mayor o menor de 0 en amount.
transaction_df_filtrado_ordenado_selected['Inflows'] = transaction_df_filtrado_ordenado_selected['amount'].apply(lambda x: x if x >= 0 else 0)
transaction_df_filtrado_ordenado_selected['Expenses'] = transaction_df_filtrado_ordenado_selected['amount'].apply(lambda x: x if x < 0 else 0)


# Drop the amount column.
transaction_df_filtrado_ordenado_selected = transaction_df_filtrado_ordenado_selected.drop(columns=['amount'])

transaction_df_filtrado_ordenado_selected

,date,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,Expenses
0,2010-01,1556,23679,48277,110153,4,0.00,-77.0
67478,2010-01,1941,18934,38609,27463,3,55.78,0.0
67477,2010-01,1797,24971,30962,15336,5,62.72,0.0
67476,2010-01,1670,32079,50184,27778,4,73.00,0.0
67475,2010-01,1357,16941,11410,0,7,0.00,-56.0
...,...,...,...,...,...,...,...,...
13227659,2019-10,349,21201,43229,96831,2,60.00,0.0
13227658,2019-10,261,28985,59093,117325,5,10.71,0.0
13227657,2019-10,201,17567,35823,96691,3,14.78,0.0
13227655,2019-10,1060,17909,21547,16040,5,14.23,0.0


In [104]:
# Group by date and client_id and sum the values of Inflows and Expenses.

transaction_df_filtrado_ordenado_selected_grouped = transaction_df_filtrado_ordenado_selected.groupby(['date', 'client_id']).sum().reset_index()

transaction_df_filtrado_ordenado_selected_grouped

,date,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,Expenses
0,2010-01,0,2777515,5663235,3438905,380,5031.38,-436.0
1,2010-01,1,1868748,3810240,1225308,252,3076.62,-557.0
2,2010-01,2,1332639,2717253,8004150,495,3149.62,-594.0
3,2010-01,3,561905,1145663,766413,164,1626.42,-170.0
4,2010-01,4,4348260,8865996,13381992,580,6305.49,-399.0
...,...,...,...,...,...,...,...,...
139362,2019-10,1994,3719465,7583460,9967623,291,5069.23,-289.0
139363,2019-10,1995,1675440,3136480,737520,320,2614.07,-315.0
139364,2019-10,1996,1317360,2686080,3591720,180,3391.19,-663.0
139365,2019-10,1997,3289020,3259200,2172828,588,7486.10,-492.0


In [112]:
# Prepare transaction_df_filtrado_ordenado_selected_grouped for prophet.

transaction_df_filtrado_ordenado_selected_grouped_prophet = transaction_df_filtrado_ordenado_selected_grouped.rename(columns={'date': 'ds', 'Expenses': 'y'})

import pandas as pd

# Convertir la columna 'ds' a formato datetime
transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'] = pd.to_datetime(transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'], format='%Y-%m')

transaction_df_filtrado_ordenado_selected_grouped_prophet



,ds,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,y
0,2010-01-01,0,2777515,5663235,3438905,380,5031.38,-436.0
1,2010-01-01,1,1868748,3810240,1225308,252,3076.62,-557.0
2,2010-01-01,2,1332639,2717253,8004150,495,3149.62,-594.0
3,2010-01-01,3,561905,1145663,766413,164,1626.42,-170.0
4,2010-01-01,4,4348260,8865996,13381992,580,6305.49,-399.0
...,...,...,...,...,...,...,...,...
139362,2019-10-01,1994,3719465,7583460,9967623,291,5069.23,-289.0
139363,2019-10-01,1995,1675440,3136480,737520,320,2614.07,-315.0
139364,2019-10-01,1996,1317360,2686080,3591720,180,3391.19,-663.0
139365,2019-10-01,1997,3289020,3259200,2172828,588,7486.10,-492.0


In [106]:
# Cuantas filas hay por año.

transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'].dt.year.value_counts()

ds
2017    14479
2018    14454
2016    14442
2015    14384
2014    14260
2013    14170
2012    14000
2011    13782
2010    13341
2019    12055
Name: count, dtype: int64

In [156]:
transaction_df_filtrado_ordenado_selected_grouped_prophet

# Filter by client_id ==1. 

transaction_df_filtrado_ordenado_selected_grouped_prophet_1 = transaction_df_filtrado_ordenado_selected_grouped_prophet[transaction_df_filtrado_ordenado_selected_grouped_prophet['client_id'] == 1]
transaction_df_filtrado_ordenado_selected_grouped_prophet_1

,ds,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,y
1,2010-01-01,1,1868748,3810240,1225308,252,3076.62,-557.0
1084,2010-02-01,1,1468302,2993760,962742,198,1731.29,0.0
2181,2010-03-01,1,1868748,3810240,1225308,252,4195.45,-655.0
3278,2010-04-01,1,1935489,3946320,1269069,261,3149.81,-618.0
4380,2010-05-01,1,1957736,3991680,1283656,264,3499.65,-168.0
...,...,...,...,...,...,...,...,...
133341,2019-06-01,1,1601784,3265920,1050264,216,2860.03,-120.0
134547,2019-07-01,1,1824254,3719520,1196134,246,3231.96,-802.0
135753,2019-08-01,1,1757513,3583440,1152373,237,2486.23,0.0
136959,2019-09-01,1,2224700,4536000,1458700,300,5800.47,-869.0


In [49]:
# Divide en train y test.
train = transaction_df_filtrado_ordenado_selected_grouped_prophet[transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'] < '2018-01-01']
test = transaction_df_filtrado_ordenado_selected_grouped_prophet[transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'] >= '2018-01-01']

In [52]:
# Lista para almacenar resultados
results = []

# Iterar sobre cada cliente único
for client_id in train['client_id'].unique():
    # Filtrar datos para el cliente actual
    client_data = train[train['client_id'] == client_id][['ds', 'y']]
    
    # Asegurarse de que hay datos suficientes
    if len(client_data) > 0:
        # Inicializar y ajustar el modelo
        model = Prophet()
        model.fit(client_data)
        
        # Crear un DataFrame para las predicciones (3 meses a partir de la última fecha en los datos)
        future = model.make_future_dataframe(periods=3, freq='M')
        
        # Hacer la predicción
        forecast = model.predict(future)
        
        # Agregar el ID del cliente a las predicciones
        forecast['client_id'] = client_id
        
        # Agregar los resultados relevantes a la lista
        results.append(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'client_id']])

17:12:00 - cmdstanpy - INFO - Chain [1] start processing
17:12:00 - cmdstanpy - INFO - Chain [1] done processing
17:12:00 - cmdstanpy - INFO - Chain [1] start processing
17:12:00 - cmdstanpy - INFO - Chain [1] done processing
17:12:00 - cmdstanpy - INFO - Chain [1] start processing
17:12:01 - cmdstanpy - INFO - Chain [1] done processing
17:12:01 - cmdstanpy - INFO - Chain [1] start processing
17:12:01 - cmdstanpy - INFO - Chain [1] done processing
17:12:01 - cmdstanpy - INFO - Chain [1] start processing
17:12:01 - cmdstanpy - INFO - Chain [1] done processing
17:12:01 - cmdstanpy - INFO - Chain [1] start processing
17:12:01 - cmdstanpy - INFO - Chain [1] done processing
17:12:01 - cmdstanpy - INFO - Chain [1] start processing
17:12:01 - cmdstanpy - INFO - Chain [1] done processing
17:12:01 - cmdstanpy - INFO - Chain [1] start processing
17:12:01 - cmdstanpy - INFO - Chain [1] done processing
17:12:01 - cmdstanpy - INFO - Chain [1] start processing
17:12:01 - cmdstanpy - INFO - Chain [1]

In [57]:
# Concatenar todos los resultados en un único DataFrame
final_results = pd.concat(results, ignore_index=True)

final_results

,ds,yhat,yhat_lower,yhat_upper,client_id
0,2010-01-01,-627.686505,-1078.085489,-176.399150,0
1,2010-02-01,-492.388426,-912.012613,-20.874927,0
2,2010-03-01,-738.216472,-1206.485275,-302.749987,0
3,2010-04-01,-421.477384,-883.822852,14.143178,0
4,2010-05-01,-573.602105,-996.852255,-130.780459,0
...,...,...,...,...,...
116510,2017-11-01,-674.969446,-1034.812749,-311.542120,22
116511,2017-12-01,-765.697140,-1110.456125,-420.977368,22
116512,2017-12-31,-856.424834,-1219.882227,-513.436546,22
116513,2018-01-31,-950.176784,-1300.904844,-587.513275,22


In [81]:
# Join the test and final_results dataframes on the columns 'ds' and 'client_id'.

# final_results['ds'] = final_results['ds'].dt.strftime('%Y-%m')
# test['ds'] = test['ds'].dt.strftime('%Y-%m')

# Merge the dataframes.

final_results_merged = final_results.merge(test, on=['ds', 'client_id'], how='left', suffixes=('_pred', '_true'))

final_results_merged

,ds,yhat,yhat_lower,yhat_upper,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,y
0,2010-01,-627.686505,-1078.085489,-176.399150,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02,-492.388426,-912.012613,-20.874927,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03,-738.216472,-1206.485275,-302.749987,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04,-421.477384,-883.822852,14.143178,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05,-573.602105,-996.852255,-130.780459,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
116510,2017-11,-674.969446,-1034.812749,-311.542120,22,NaN,NaN,NaN,NaN,NaN,NaN
116511,2017-12,-765.697140,-1110.456125,-420.977368,22,NaN,NaN,NaN,NaN,NaN,NaN
116512,2017-12,-856.424834,-1219.882227,-513.436546,22,NaN,NaN,NaN,NaN,NaN,NaN
116513,2018-01,-950.176784,-1300.904844,-587.513275,22,2990771.0,6098158.0,15309014.0,113.0,3552.55,-154.0


In [82]:
# Drop rows with NaN values.

final_results_merged_cleaned = final_results_merged.dropna()

final_results_merged_cleaned

,ds,yhat,yhat_lower,yhat_upper,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,y
97,2018-01,-708.180398,-1158.464247,-231.373795,0,4093180.0,8345820.0,5067860.0,560.0,8667.07,-1192.0
98,2018-02,-511.231298,-936.253148,-25.367535,0,3069885.0,6259365.0,3800895.0,420.0,4694.84,-955.0
196,2018-01,-25.795650,-188.343147,130.833456,1,1735266.0,3538080.0,1137786.0,234.0,2228.22,0.0
197,2018-02,-280.072016,-434.173145,-118.910127,1,1735266.0,3538080.0,1137786.0,234.0,2604.62,-75.0
295,2018-01,-21.252120,-234.855748,197.110516,2,1332639.0,2717253.0,8004150.0,495.0,3523.99,-547.0
...,...,...,...,...,...,...,...,...,...,...,...
116492,2018-02,-107.698395,-390.966088,168.709852,1862,1912504.0,3899992.0,5876356.0,103.0,3120.62,0.0
116504,2018-01,-1491.608439,-2073.229029,-917.682776,1527,2354005.0,4799685.0,9382485.0,95.0,3907.24,-509.0
116505,2018-02,-1587.480059,-2164.319142,-1002.971192,1527,1709751.0,3486087.0,6814647.0,69.0,4648.98,-717.0
116513,2018-01,-950.176784,-1300.904844,-587.513275,22,2990771.0,6098158.0,15309014.0,113.0,3552.55,-154.0


In [83]:
from sklearn.metrics import r2_score

r2 = r2_score(final_results_merged_cleaned['y'], final_results_merged_cleaned['yhat'])
print(f'R²: {r2:.4f}')

R²: 0.3651


Ahora con el total

In [113]:
transaction_df_filtrado_ordenado_selected_grouped_prophet

,ds,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,y
0,2010-01-01,0,2777515,5663235,3438905,380,5031.38,-436.0
1,2010-01-01,1,1868748,3810240,1225308,252,3076.62,-557.0
2,2010-01-01,2,1332639,2717253,8004150,495,3149.62,-594.0
3,2010-01-01,3,561905,1145663,766413,164,1626.42,-170.0
4,2010-01-01,4,4348260,8865996,13381992,580,6305.49,-399.0
...,...,...,...,...,...,...,...,...
139362,2019-10-01,1994,3719465,7583460,9967623,291,5069.23,-289.0
139363,2019-10-01,1995,1675440,3136480,737520,320,2614.07,-315.0
139364,2019-10-01,1996,1317360,2686080,3591720,180,3391.19,-663.0
139365,2019-10-01,1997,3289020,3259200,2172828,588,7486.10,-492.0


In [ ]:
transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'] = transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'].dt.strftime('%Y-%m')


In [133]:
test_92 = transaction_df_filtrado_ordenado_selected_grouped_prophet[transaction_df_filtrado_ordenado_selected_grouped_prophet['client_id'] == 92] 
test_92.ds.max()

'2019-10'

In [142]:
# Convertir la columna 'ds' a datetime si no lo está ya
transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'] = pd.to_datetime(transaction_df_filtrado_ordenado_selected_grouped_prophet['ds'])

# Lista para almacenar resultados
results = []

# Iterar sobre cada cliente único
for client_id in transaction_df_filtrado_ordenado_selected_grouped_prophet['client_id'].unique():
    # Filtrar datos para el cliente actual
    client_data = transaction_df_filtrado_ordenado_selected_grouped_prophet[transaction_df_filtrado_ordenado_selected_grouped_prophet['client_id'] == client_id][['ds', 'y']]
    
    # Asegurarse de que hay datos suficientes
    if len(client_data) > 0:
        # Inicializar y ajustar el modelo
        model = Prophet()
        model.fit(client_data)
        
        # Crear un DataFrame para las predicciones (3 meses a partir de la última fecha en los datos)
        future = model.make_future_dataframe(periods=4, freq='M')
        
        # Hacer la predicción
        forecast = model.predict(future)
        
        # Agregar el ID del cliente a las predicciones
        forecast['client_id'] = client_id
        
        # Filtrar solo las predicciones para los próximos 3 meses
        last_date = client_data['ds'].max() + pd.DateOffset(months=1)  # Aumentar un mes a la última fecha
        
        # Filtrar las fechas futuras
        filtered_forecast = forecast[forecast['ds'] > last_date]  # Filtrar fechas futuras
        
        # Asegurarse de que solo se seleccionen los 3 meses siguientes
        filtered_forecast = filtered_forecast.head(3)  # Mantener solo las 3 filas
        
        # Agregar los resultados relevantes a la lista
        results.append(filtered_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'client_id']])

# Concatenar los resultados en un solo DataFrame
final_results = pd.concat(results, ignore_index=True)

# Mostrar los resultados finales
print(final_results)

17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1] done processing
17:55:31 - cmdstanpy - INFO - Chain [1] start processing
17:55:31 - cmdstanpy - INFO - Chain [1]

             ds         yhat   yhat_lower   yhat_upper  client_id
0    2019-11-30  -841.007466 -1280.037914  -337.504230          0
1    2019-12-31  -704.755426 -1177.242810  -208.523193          0
2    2020-01-31  -681.585500 -1150.903501  -187.769146          0
3    2019-11-30   -89.419600  -286.643986   110.950800          1
4    2019-12-31  -121.640061  -326.170518    86.795556          1
...         ...          ...          ...          ...        ...
3652 2019-12-31 -3405.907866 -3767.070859 -3019.497792       1527
3653 2020-01-31  1878.046809  1521.087369  2248.939083       1527
3654 2019-11-30   645.456505   508.140384   776.316311         22
3655 2019-12-31   608.122093   474.505464   743.161296         22
3656 2020-01-31 -3185.496430 -3319.241601 -3040.249125         22

[3657 rows x 5 columns]


In [151]:
final_results

final_results['ds'] = final_results['ds'].dt.strftime('%Y-%m')

final_results

AttributeError: Can only use .dt accessor with datetimelike values

In [148]:
# in final_results, if yhat is positive, then yhat = 0.

final_results['yhat'] = final_results['yhat'].apply(lambda x: x if x < 0 else 0)

final_results

,ds,yhat,yhat_lower,yhat_upper,client_id
0,2019-11,-841.007466,-1280.037914,-337.504230,0
1,2019-12,-704.755426,-1177.242810,-208.523193,0
2,2020-01,-681.585500,-1150.903501,-187.769146,0
3,2019-11,-89.419600,-286.643986,110.950800,1
4,2019-12,-121.640061,-326.170518,86.795556,1
...,...,...,...,...,...
3652,2019-12,-3405.907866,-3767.070859,-3019.497792,1527
3653,2020-01,0.000000,1521.087369,2248.939083,1527
3654,2019-11,0.000000,508.140384,776.316311,22
3655,2019-12,0.000000,474.505464,743.161296,22


In [153]:
final_results

,ds,yhat,yhat_lower,yhat_upper,client_id
0,2019-11,-841.007466,-1280.037914,-337.504230,0
1,2019-12,-704.755426,-1177.242810,-208.523193,0
2,2020-01,-681.585500,-1150.903501,-187.769146,0
3,2019-11,-89.419600,-286.643986,110.950800,1
4,2019-12,-121.640061,-326.170518,86.795556,1
...,...,...,...,...,...
3652,2019-12,-3405.907866,-3767.070859,-3019.497792,1527
3653,2020-01,0.000000,1521.087369,2248.939083,1527
3654,2019-11,0.000000,508.140384,776.316311,22
3655,2019-12,0.000000,474.505464,743.161296,22


In [154]:
import json

# Crear el diccionario para almacenar los resultados en el formato deseado
export_data = {"target": {}}

# Iterar sobre cada fila en final_results
for _, row in final_results.iterrows():
    client_id = str(row['client_id'])  # Asegurarse de que el client_id sea una cadena
    date_key = row['ds']  # Formatear la fecha como 'YYYY-MM'
    value = row['yhat']  # Obtener el valor de la predicción (o el que desees)

    # Inicializar el diccionario para el cliente si no existe
    if client_id not in export_data["target"]:
        export_data["target"][client_id] = {}

    # Agregar el valor al diccionario del cliente
    export_data["target"][client_id][date_key] = value

# Exportar a un archivo JSON
with open('predictions.json', 'w') as json_file:
    json.dump(export_data, json_file, indent=4)

# Mostrar el contenido del diccionario exportado (opcional)
print(json.dumps(export_data, indent=4))

{
    "target": {
        "0": {
            "2019-11": -841.0074658868117,
            "2019-12": -704.7554257130022,
            "2020-01": -681.5855003363102
        },
        "1": {
            "2019-11": -89.41959996134486,
            "2019-12": -121.64006120578901,
            "2020-01": -40.004119799487796
        },
        "2": {
            "2019-11": -280.5021937047265,
            "2019-12": -323.55645998754443,
            "2020-01": -264.51037225371545
        },
        "3": {
            "2019-11": -205.19615735267143,
            "2019-12": -103.91090451321355,
            "2020-01": -110.11576099954841
        },
        "4": {
            "2019-11": -555.6271031887011,
            "2019-12": -380.6127385788543,
            "2020-01": -541.2926955550739
        },
        "5": {
            "2019-11": -251.80489174352655,
            "2019-12": -328.33273027332444,
            "2020-01": -256.7948294932508
        },
        "11": {
            "2019-11": -206.30373

In [149]:
# final_results_total filtrado por cliente 116.
final_results[final_results['client_id'] == 1957]

,ds,yhat,yhat_lower,yhat_upper,client_id
3159,2013-02,-457.240986,-687.968853,-248.143848,1957
3160,2013-03,-638.722109,-851.117134,-397.606983,1957
3161,2013-04,-763.041134,-972.114535,-514.141991,1957


In [127]:
transaction_df_filtrado_ordenado_selected_grouped_prophet[transaction_df_filtrado_ordenado_selected_grouped_prophet['client_id'] == 92]

,ds,client_id,per_capita_income,yearly_income,total_debt,num_credit_cards,Inflows,y
54,2010-01-01,92,2256523,3332515,549124,428,3041.95,-195.0
1137,2010-02-01,92,2003455,2958775,487540,380,4572.73,-559.0
2234,2010-03-01,92,1940188,2865340,472144,368,2935.31,0.0
3331,2010-04-01,92,1834743,2709615,446484,348,3464.12,-606.0
4433,2010-05-01,92,1876921,2771905,456748,356,3067.34,-493.0
...,...,...,...,...,...,...,...,...
133399,2019-06-01,92,2045633,3021065,497804,388,3751.15,-144.0
134605,2019-07-01,92,2066722,3052210,502936,392,3381.35,-159.0
135811,2019-08-01,92,2151078,3176790,523464,408,3612.61,-574.0
137017,2019-09-01,92,2151078,3176790,523464,408,3001.94,-151.0


In [45]:
transaction_df_filtrado_ordenado_selected_grouped_prophet.dtypes

ds                    object
client_id              int64
per_capita_income      int64
yearly_income          int64
total_debt             int64
num_credit_cards       int64
Inflows              float64
y                    float64
dtype: object

In [ ]:
Formato_modelo= transaction_df_filtrado_ordenado_selected_grouped